<a href="https://colab.research.google.com/github/medinadiegoeverardo/DS-Unit-2-Regression-Classification/blob/master/module3/medinadiego_3_assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `RidgeRegression`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `RidgeRegression`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 156, done.
remote: Total 156 (delta 0), reused 0 (delta 0), pack-reused 156
Receiving objects: 100% (156/156), 19.30 MiB | 19.88 MiB/s, done.
Resolving deltas: 100% (71/71), done.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
 * [new branch]      master     -> origin/master
     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 112kB 8.8MB/s 
     |████████████████████████████████| 133kB 44.8MB/s 
     |████████████████████████████████| 57.7MB 536kB/s 
     |████████████████████████████████| 235kB 42.2MB/s 
     |████████████████████████████████| 614kB 30.0MB/s 
     |████████████████████████████████| 235kB 47.7MB/s 
     |████████████████████████████████| 307kB 51.1MB/s 
     |████████████████████████████████| 215kB 36.4MB/s 
     |████████████████████████████████| 51kB 23.8MB/s 
     |██████████

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
df_copy = df.copy() # for Ridge Regression

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER' # All values in neighborhood that are not in top10, 'OTHER'

In [0]:
# how would selecting data for less cardinality randomly, instead of selecting the top 10 or 20 like above affect the regression results?

import random
random_neighbor = random.sample(list(df['NEIGHBORHOOD']), 30)

In [0]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [0]:
df.shape

(23040, 21)

In [0]:
# df['SALE_PRICE'].sort_values(ascending=False)

In [0]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,23040.0,4.459116e+03,3.713270e+03,1.0,1341.0,3546.0,6673.75,16350.0
LOT,23040.0,3.539485e+02,6.280253e+02,1.0,22.0,49.0,375.00,9057.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,23039.0,1.079119e+04,1.103414e+03,0.0,10306.0,11211.0,11360.00,11697.0
RESIDENTIAL_UNITS,23039.0,1.827423e+00,1.009043e+01,0.0,0.0,1.0,2.00,750.0
COMMERCIAL_UNITS,23039.0,2.865142e-01,5.778224e+00,-148.0,0.0,0.0,0.00,570.0
TOTAL_UNITS,23039.0,2.320891e+00,1.207055e+01,0.0,1.0,1.0,2.00,755.0
GROSS_SQUARE_FEET,23039.0,3.508555e+03,2.218610e+04,0.0,494.0,1356.0,2280.00,1303935.0
YEAR_BUILT,23005.0,1.819672e+03,4.883766e+02,0.0,1920.0,1940.0,1965.00,2019.0
TAX_CLASS_AT_TIME_OF_SALE,23040.0,1.624479e+00,8.038978e-01,1.0,1.0,1.0,2.00,4.0


In [0]:
df.describe(include=['O']).T

,count,unique,top,freq
BOROUGH,23040,5,4,7494
NEIGHBORHOOD,23040,11,OTHER,19117
BUILDING_CLASS_CATEGORY,23040,44,01 ONE FAMILY DWELLINGS,5061
TAX_CLASS_AT_PRESENT,23039,10,1,11071
BUILDING_CLASS_AT_PRESENT,23039,146,D4,3408
ADDRESS,23040,22691,100 JEROME STREET,6
APARTMENT_NUMBER,5201,1724,4,99
LAND_SQUARE_FEET,22987,3652,0,7500
BUILDING_CLASS_AT_TIME_OF_SALE,23040,147,D4,3408
SALE_DATE,23040,120,01/24/2019,480


### fillna those with little null values, convert some values to int and dates, reducing cardinality on other columns

In [0]:
features_nulls_high_cardinality = ['APARTMENT_NUMBER', 'EASE-MENT', 'ADDRESS', 'BUILDING_CLASS_AT_PRESENT']
df = df.drop(features_nulls_high_cardinality, axis=1)

In [0]:
top_land_sqft = df['LAND_SQUARE_FEET'].value_counts()[:35].index

# how isin works

# creating a bool series from isin() 
# new = data["Gender"].isin(["Male"]) 
# displaying data with gender = male only 
# data[new]

# a filter, in other words

df.loc[~df['LAND_SQUARE_FEET'].isin(top_land_sqft), 'LAND_SQUARE_FEET'] = 'N/A'

building_class_top = df['BUILDING_CLASS_CATEGORY'].value_counts()[:30].index
df.loc[~df['BUILDING_CLASS_CATEGORY'].isin(building_class_top), 'BUILDING_CLASS_CATEGORY'] = 'N/A'

In [0]:
# reduced neighborhood (252 to 10), land square feet (22987 to 35)

In [0]:
def filling_nulls(df):
  columns = df.columns
  for col in columns:
    if df[col].isnull().sum() > 0:
      if df[col].dtypes == int or float:
        df[col].fillna(method='ffill', inplace=True)
      else:
        pass

filling_nulls(df)

In [0]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'])

## Splitting

In [0]:
# January — March 2019 to train. Use data from April 2019 to test.

In [0]:
# from sklearn.model_selection import train_test_split
# target_y = df['SALE_PRICE']
# features_x = X = df.drop('SALE_PRICE', axis='columns')
# x_train, x_test, y_train, y_test = train_test_split(features_x, target_y, test_size=.5, random_state=42)

In [0]:
df['SALE_DATE'].describe()

count                   23040
unique                    120
top       2019-01-24 00:00:00
freq                      480
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [0]:
# cutoff

# df[(df['created'] > '2016-04-01') & (df['created'] <= '2016-05-30')]
#train = df[(df['SALE_DATE'] > '2019-01-01') & (df['SALE_DATE'] < '2019-03-31')]

cutoff = '2019-03-31'
train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] > cutoff]

In [0]:
train.tail() # Jan to March

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18148,4,OTHER,02 TWO FAMILY DWELLINGS,1,2109,19,11374.0,2.0,0.0,2.0,N/A,1368.0,1935.0,1,B3,0,2019-03-30
18149,4,OTHER,02 TWO FAMILY DWELLINGS,1,9280,23,11418.0,2.0,0.0,2.0,N/A,2160.0,1925.0,1,B2,0,2019-03-30
18150,4,OTHER,02 TWO FAMILY DWELLINGS,1,9518,17,11419.0,2.0,0.0,2.0,N/A,2248.0,1920.0,1,B2,0,2019-03-30
18151,4,OTHER,02 TWO FAMILY DWELLINGS,1,4643,15,11357.0,2.0,0.0,2.0,"2,500",1396.0,1935.0,1,B1,0,2019-03-30
18152,5,OTHER,01 ONE FAMILY DWELLINGS,1,2005,107,10314.0,1.0,0.0,1.0,N/A,1944.0,1988.0,1,A5,0,2019-03-30


In [0]:
train.shape

(18153, 17)

In [0]:
train.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                          object
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE                         datetime64[ns]
dtype: object

In [0]:
test.tail() # April

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
23035,4,OTHER,01 ONE FAMILY DWELLINGS,1,10965,276,11429.0,1.0,0.0,1.0,"1,800",1224.0,1945.0,1,A5,510000,2019-04-30
23036,4,OTHER,09 COOPS - WALKUP APARTMENTS,2,169,29,11104.0,0.0,0.0,0.0,0,0.0,1929.0,2,C6,355000,2019-04-30
23037,4,OTHER,10 COOPS - ELEVATOR APARTMENTS,2,131,4,11377.0,0.0,0.0,0.0,0,0.0,1932.0,2,D4,375000,2019-04-30
23038,4,OTHER,02 TWO FAMILY DWELLINGS,1,8932,18,11421.0,2.0,1.0,3.0,N/A,2200.0,1931.0,1,S2,1100000,2019-04-30
23039,4,OTHER,12 CONDOS - WALKUP APARTMENTS,2,1216,1161,11377.0,1.0,0.0,85.0,N/A,854.0,1927.0,2,R2,569202,2019-04-30


### Baselines

In [0]:
# mean_absolute_error function needs 2 inputs that are the same length. 1 input should be the 
# average y_variable (predicted baseline) and the other input is the actual y_variable from sets(both same length)

In [0]:
import numpy as np
from sklearn.metrics import mean_absolute_error

# getting_metrics_training(train['SALE_PRICE'], test['SALE_PRICE'])

baseline_train = np.mean(train['SALE_PRICE'])
print(f'Predicted (mean) price is: ${baseline_train:,.2f}')

  # Baseline (1 feature)

  # Mean absolute error for Training
y_train = train['SALE_PRICE'] # series of training target
y_train_pred = [baseline_train] * len(y_train) # this step is necessary for function to work (same length) - this is our target repeated (the predicted value)
mae_train = mean_absolute_error(y_train, y_train_pred) 

Predicted (mean) price is: $1,218,269.95


In [0]:
# for mean and 1 feature baseline

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

def getting_metrics_training(target):

  # Mean initial baseline

  baseline_train = np.mean(target)
  print(f'Predicted (mean) price is: ${baseline_train:,.2f}')

  # Baseline (1 feature)

  # Mean absolute error for Training
  y_train = target # series of training target
  y_train_pred = [baseline_train] * len(y_train) # this step is necessary for function to work (same length) - this is our target repeated (the predicted value)
  mae_train = mean_absolute_error(y_train, y_train_pred) 
  print(f"Our Training model's MA error is: ${mae_train:,.0f}")

  # Mean squared error for Training
  squared_train_error = np.sqrt(mean_squared_error(y_train, y_train_pred)) # same input as with MAE, diff func
  print(f"Our Training model's squared error is: ${squared_train_error:,.0f}")

  # r^2 score for Training
  r_training_score = r2_score(y_train, y_train_pred) # same input as with MAE and MSE, diff func
  print('R-score Training error is: {0:.4f}'.format(r_training_score))


  # # Mean absolute error for Testing
  # y_train_pred = [baseline_train] * len(y_test)
  # mae = mean_absolute_error(y_test, y_train_pred)
  # print(f'Our Testing model\'s MA error is: ${mae:,.2f}')

  # # Mean squared error for Testing
  # squared_train_error = np.sqrt(mean_squared_error(y_test, y_train_pred)) # same input as with MAE, diff func
  # print(f"Our Testing model's squared error is: ${squared_train_error:,.0f}")

  # # r^2 score for Testing
  # r_training_score = r2_score(y_test, y_train_pred) # same input as with MAE and MSE, diff func
  # print('R-score Testing error is: {0:.4f}'.format(r_training_score))

getting_metrics_training(train['SALE_PRICE'])

Predicted (mean) price is: $1,218,269.95
Our Training model's MA error is: $1,468,452
Our Training model's squared error is: $10,925,076
R-score Training error is: 0.0000


In [0]:
import pandas as pd
import plotly.express as px

px.scatter(df, x='GROSS_SQUARE_FEET', y='SALE_PRICE', trendline='ols')

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning:

Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.



In [0]:
# 1 feature baseline for training and testing

from sklearn.linear_model import LinearRegression

def fitting_predicting(train, test, features, target):
  
  model = LinearRegression()
  
  # getting matrix and vector ready

  # features = ['GROSS_SQUARE_FEET']
  #try:
    
    # this syntax only works when the target and features are all in two dataframes only (train and test). Once we split
    # for x_train, y_train, x_test, y_test, we have the similar inputs but syntax needs to change for that reason

  # try is for when target and features are still in train/test sets
  y_train = train[target]
  y_test = test[target]
    #target = 'SALE_PRICE'

  x_train = train[features]
  x_test = test[features]
  print(f'Linear Regression, dependent on: {features}')
    
  # except KeyError:
      
  #     y_train = [target]
  #     y_test = [target]

  #     x_train = features
  #     x_test = features
  #     print(f'Linear Regression, dependent on: {list(features)}')

  # Training error
  model.fit(x_train, y_train)
  y_pred = model.predict(x_train)
  train_mae = mean_absolute_error(y_train, y_pred)
  print('Training MAE: ${0:,.2f}'.format(train_mae))

  # Testing error
  y_pred = model.predict(x_test)
  testma_error = mean_absolute_error(y_test, y_pred)
  print('Testing MAE: ${0:,.2f}'.format(testma_error))
  print(f'Coefficient {model.coef_}, intercept: {model.intercept_}')

In [0]:
fitting_predicting(train, test, ['GROSS_SQUARE_FEET'], ['SALE_PRICE'])

Linear Regression, dependent on: ['GROSS_SQUARE_FEET']
Training MAE: $1,352,643.30
Testing MAE: $2,095,710.90
Coefficient [[270.28567279]], intercept: [349198.96667484]


In [0]:
# fitting_predicting(x_train, x_test, x_train_select_features, y_train) # y_train is the SALE_PRICE    (COMING BACK TO THIS)

### Encoding/feature engineering

In [0]:
target = 'SALE_PRICE'
x_train = train.drop(columns=['SALE_PRICE'])
y_train = train[target]
x_test = test.drop(columns=['SALE_PRICE'])
y_test = test[target]

In [0]:
# before encoding (18153, 20)
x_train.shape

(18153, 16)

In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
x_train = encoder.fit_transform(x_train) # Fit to data, then transform it.
x_test = encoder.transform(x_test) # applying FE to both

In [0]:
x_train.head()

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_UPPER WEST SIDE (59-79),NEIGHBORHOOD_UPPER WEST SIDE (79-96),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_GRAMERCY,BUILDING_CLASS_CATEGORY_13 CONDOS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_15 CONDOS - 2-10 UNIT RESIDENTIAL,BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_05 TAX CLASS 1 VACANT LAND,BUILDING_CLASS_CATEGORY_29 COMMERCIAL GARAGES,BUILDING_CLASS_CATEGORY_N/A,BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_31 COMMERCIAL VACANT LAND,BUILDING_CLASS_CATEGORY_41 TAX CLASS 4 - OTHER,BUILDING_CLASS_CATEGORY_14 RENTALS - 4-10 UNIT,BUILDING_CLASS_CATEGORY_03 THREE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS,BUILDING_CLASS_CATEGORY_10 COOPS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_44 CONDO PARKING,BUILDING_CLASS_CATEGORY_47 CONDO NON-BUSINESS STORAGE,BUILDING_CLASS_CATEGORY_12 CONDOS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_43 CONDO OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_09 COOPS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_27 FACTORIES,BUILDING_CLASS_CATEGORY_04 TAX CLASS 1 CONDOS,BUILDING_CLASS_CATEGORY_17 CONDO COOPS,...,BUILDING_CLASS_AT_TIME_OF_SALE_Q9,BUILDING_CLASS_AT_TIME_OF_SALE_M3,BUILDING_CLASS_AT_TIME_OF_SALE_S0,BUILDING_CLASS_AT_TIME_OF_SALE_Z2,BUILDING_CLASS_AT_TIME_OF_SALE_L9,BUILDING_CLASS_AT_TIME_OF_SALE_E7,BUILDING_CLASS_AT_TIME_OF_SALE_D8,BUILDING_CLASS_AT_TIME_OF_SALE_J9,BUILDING_CLASS_AT_TIME_OF_SALE_E2,BUILDING_CLASS_AT_TIME_OF_SALE_W8,BUILDING_CLASS_AT_TIME_OF_SALE_R5,BUILDING_CLASS_AT_TIME_OF_SALE_HB,BUILDING_CLASS_AT_TIME_OF_SALE_H9,BUILDING_CLASS_AT_TIME_OF_SALE_HR,BUILDING_CLASS_AT_TIME_OF_SALE_K3,BUILDING_CLASS_AT_TIME_OF_SALE_I4,BUILDING_CLASS_AT_TIME_OF_SALE_G8,BUILDING_CLASS_AT_TIME_OF_SALE_H7,BUILDING_CLASS_AT_TIME_OF_SALE_G9,BUILDING_CLASS_AT_TIME_OF_SALE_I5,BUILDING_CLASS_AT_TIME_OF_SALE_V9,BUILDING_CLASS_AT_TIME_OF_SALE_RT,BUILDING_CLASS_AT_TIME_OF_SALE_P8,BUILDING_CLASS_AT_TIME_OF_SALE_Z4,BUILDING_CLASS_AT_TIME_OF_SALE_L8,BUILDING_CLASS_AT_TIME_OF_SALE_P2,BUILDING_CLASS_AT_TIME_OF_SALE_L1,BUILDING_CLASS_AT_TIME_OF_SALE_V3,BUILDING_CLASS_AT_TIME_OF_SALE_T2,BUILDING_CLASS_AT_TIME_OF_SALE_RA,BUILDING_CLASS_AT_TIME_OF_SALE_G4,BUILDING_CLASS_AT_TIME_OF_SALE_V2,BUILDING_CLASS_AT_TIME_OF_SALE_H2,BUILDING_CLASS_AT_TIME_OF_SALE_P9,BUILDING_CLASS_AT_TIME_OF_SALE_W4,BUILDING_CLASS_AT_TIME_OF_SALE_W3,BUILDING_CLASS_AT_TIME_OF_SALE_I7,BUILDING_CLASS_AT_TIME_OF_SALE_H4,BUILDING_CLASS_AT_TIME_OF_SALE_K6,SALE_DATE
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01
4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019-01-01


In [0]:
x_train.shape # after (18153, 243)

(18153, 243)

In [0]:
# all_types = [x for x in x_train.dtypes]

# [x_train.dtypes]

# for i,j in x_train.dtypes:
#   count = []
#   keypa = {}
#   if i, j == dtype('Timestamp'):
#     count += 1
#     keypa.update()

### SelectKBest

In [0]:
# dtyp = x_train.dtypes == 'M8[ns]'
# (x_train.dtypes == 'M8[ns]')
import numpy as np

x_train['SALE_DATE'] = x_train['SALE_DATE'].astype(np.int64) // 10**9
x_test['SALE_DATE'] = x_test['SALE_DATE'].astype(np.int64) // 10**9

In [0]:
# select kbest does not take in datetimes. considered dropping (error: float() argument must be a string or a number, not 'Timestamp')

# x_train= x_train.drop(columns=['SALE_DATE'])
# x_test = x_test.drop(columns=['SALE_DATE'])

In [0]:
x_train.head()

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,NEIGHBORHOOD_OTHER,NEIGHBORHOOD_UPPER EAST SIDE (59-79),NEIGHBORHOOD_UPPER EAST SIDE (79-96),NEIGHBORHOOD_BOROUGH PARK,NEIGHBORHOOD_ASTORIA,NEIGHBORHOOD_FOREST HILLS,NEIGHBORHOOD_UPPER WEST SIDE (59-79),NEIGHBORHOOD_UPPER WEST SIDE (79-96),NEIGHBORHOOD_BEDFORD STUYVESANT,NEIGHBORHOOD_FLUSHING-NORTH,NEIGHBORHOOD_GRAMERCY,BUILDING_CLASS_CATEGORY_13 CONDOS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_15 CONDOS - 2-10 UNIT RESIDENTIAL,BUILDING_CLASS_CATEGORY_07 RENTALS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_05 TAX CLASS 1 VACANT LAND,BUILDING_CLASS_CATEGORY_29 COMMERCIAL GARAGES,BUILDING_CLASS_CATEGORY_N/A,BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_31 COMMERCIAL VACANT LAND,BUILDING_CLASS_CATEGORY_41 TAX CLASS 4 - OTHER,BUILDING_CLASS_CATEGORY_14 RENTALS - 4-10 UNIT,BUILDING_CLASS_CATEGORY_03 THREE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS,BUILDING_CLASS_CATEGORY_10 COOPS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_44 CONDO PARKING,BUILDING_CLASS_CATEGORY_47 CONDO NON-BUSINESS STORAGE,BUILDING_CLASS_CATEGORY_12 CONDOS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_43 CONDO OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_09 COOPS - WALKUP APARTMENTS,BUILDING_CLASS_CATEGORY_27 FACTORIES,BUILDING_CLASS_CATEGORY_04 TAX CLASS 1 CONDOS,BUILDING_CLASS_CATEGORY_17 CONDO COOPS,...,BUILDING_CLASS_AT_TIME_OF_SALE_Q9,BUILDING_CLASS_AT_TIME_OF_SALE_M3,BUILDING_CLASS_AT_TIME_OF_SALE_S0,BUILDING_CLASS_AT_TIME_OF_SALE_Z2,BUILDING_CLASS_AT_TIME_OF_SALE_L9,BUILDING_CLASS_AT_TIME_OF_SALE_E7,BUILDING_CLASS_AT_TIME_OF_SALE_D8,BUILDING_CLASS_AT_TIME_OF_SALE_J9,BUILDING_CLASS_AT_TIME_OF_SALE_E2,BUILDING_CLASS_AT_TIME_OF_SALE_W8,BUILDING_CLASS_AT_TIME_OF_SALE_R5,BUILDING_CLASS_AT_TIME_OF_SALE_HB,BUILDING_CLASS_AT_TIME_OF_SALE_H9,BUILDING_CLASS_AT_TIME_OF_SALE_HR,BUILDING_CLASS_AT_TIME_OF_SALE_K3,BUILDING_CLASS_AT_TIME_OF_SALE_I4,BUILDING_CLASS_AT_TIME_OF_SALE_G8,BUILDING_CLASS_AT_TIME_OF_SALE_H7,BUILDING_CLASS_AT_TIME_OF_SALE_G9,BUILDING_CLASS_AT_TIME_OF_SALE_I5,BUILDING_CLASS_AT_TIME_OF_SALE_V9,BUILDING_CLASS_AT_TIME_OF_SALE_RT,BUILDING_CLASS_AT_TIME_OF_SALE_P8,BUILDING_CLASS_AT_TIME_OF_SALE_Z4,BUILDING_CLASS_AT_TIME_OF_SALE_L8,BUILDING_CLASS_AT_TIME_OF_SALE_P2,BUILDING_CLASS_AT_TIME_OF_SALE_L1,BUILDING_CLASS_AT_TIME_OF_SALE_V3,BUILDING_CLASS_AT_TIME_OF_SALE_T2,BUILDING_CLASS_AT_TIME_OF_SALE_RA,BUILDING_CLASS_AT_TIME_OF_SALE_G4,BUILDING_CLASS_AT_TIME_OF_SALE_V2,BUILDING_CLASS_AT_TIME_OF_SALE_H2,BUILDING_CLASS_AT_TIME_OF_SALE_P9,BUILDING_CLASS_AT_TIME_OF_SALE_W4,BUILDING_CLASS_AT_TIME_OF_SALE_W3,BUILDING_CLASS_AT_TIME_OF_SALE_I7,BUILDING_CLASS_AT_TIME_OF_SALE_H4,BUILDING_CLASS_AT_TIME_OF_SALE_K6,SALE_DATE
0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1546300800
1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1546300800
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1546300800
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1546300800
4,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1546300800


In [0]:
y_train = y_train.to_frame()

In [0]:
# import matplotlib.pyplot as plt

# x=y_train['gross_square_ft']
# y=y_train['SALE_PRICE']
# plt.scatter(x, y)

In [0]:
from sklearn.feature_selection import f_regression, SelectKBest


# select kbest does not take datetimes as I mentioned above. tried converting
# datetime to int and then tried dropping sale dates altogether to see if output changed
# results are the same with the date as int or without dates at all

for k in range(1, len(x_train.columns)-200): # I feel like SelectKBest would work better if the columns were not encoded? 

  print (f'{k} feature(s)')

  selector = SelectKBest(score_func=f_regression, k=k)
  train_select = selector.fit_transform(x_train, y_train)
  test_select = selector.transform(x_test) # cannot take datetimes, converted them to int above

  model_1 = LinearRegression()

  model_1.fit(train_select, y_train)
  y_pred = model_1.predict(test_select)

  mae_0 = mean_absolute_error(y_test, y_pred)
  print(f'MAE: {mae_0:,.2f}')

  # it looks like the more features the worse MAE..
  # 7 feature(s) yield the best MAE: 1,788,537.14

#   1 feature(s)
# MAE: 1,782,043.81
# 2 feature(s)
# MAE: 1,966,821.53
# 3 feature(s)
# MAE: 1,878,071.01
# 4 feature(s)
# MAE: 1,813,947.10

1 feature(s)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



NameError: ignored

In [0]:
# first 7 features gave the best MAE

all_names = x_train.columns
mask = selector.get_support()
selected_names = all_names[mask]
non_select_names = all_names[~mask]

for name in selected_names:
  print(name)

print('\n')
#for name in non_select_names:
  #print(name)

BUILDING_CLASS_AT_TIME_OF_SALE_K3




### Other SelectKBEST metrics

In [0]:
x_train_select_features = x_train[selected_names]
# x_test_select_features = x_train[selected_names]

In [0]:
x_train_select_features.head()

,BOROUGH_1,BOROUGH_4,BOROUGH_5,NEIGHBORHOOD_UPPER EAST SIDE (59-79),BUILDING_CLASS_CATEGORY_21 OFFICE BUILDINGS,BUILDING_CLASS_CATEGORY_01 ONE FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_02 TWO FAMILY DWELLINGS,BUILDING_CLASS_CATEGORY_N/A,BUILDING_CLASS_CATEGORY_08 RENTALS - ELEVATOR APARTMENTS,BUILDING_CLASS_CATEGORY_22 STORE BUILDINGS,BUILDING_CLASS_CATEGORY_45 CONDO HOTELS,BUILDING_CLASS_CATEGORY_30 WAREHOUSES,BUILDING_CLASS_CATEGORY_11A CONDO-RENTALS,TAX_CLASS_AT_PRESENT_4,TAX_CLASS_AT_PRESENT_1,BLOCK,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET_N/A,GROSS_SQUARE_FEET,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_O5,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_I9,BUILDING_CLASS_AT_TIME_OF_SALE_D6,BUILDING_CLASS_AT_TIME_OF_SALE_RH,BUILDING_CLASS_AT_TIME_OF_SALE_E1,BUILDING_CLASS_AT_TIME_OF_SALE_D1,BUILDING_CLASS_AT_TIME_OF_SALE_O4,BUILDING_CLASS_AT_TIME_OF_SALE_E9,BUILDING_CLASS_AT_TIME_OF_SALE_O6,BUILDING_CLASS_AT_TIME_OF_SALE_H1,BUILDING_CLASS_AT_TIME_OF_SALE_O2,BUILDING_CLASS_AT_TIME_OF_SALE_RR,BUILDING_CLASS_AT_TIME_OF_SALE_D8,BUILDING_CLASS_AT_TIME_OF_SALE_W8,BUILDING_CLASS_AT_TIME_OF_SALE_R5,BUILDING_CLASS_AT_TIME_OF_SALE_K3,BUILDING_CLASS_AT_TIME_OF_SALE_I4,BUILDING_CLASS_AT_TIME_OF_SALE_H2
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,716,10011.0,1.0,0.0,1.0,1,1979.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,812,10018.0,0.0,6.0,6.0,1,15435.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,839,10018.0,0.0,7.0,7.0,1,11332.0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,592,10014.0,1.0,0.0,1.0,0,500.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1379,10065.0,1.0,0.0,1.0,0,6406.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# target = 'SALE_PRICE'
# x_train = train.drop(columns=['SALE_PRICE'])
# y_train = train[target]
# x_test = test.drop(columns=['SALE_PRICE'])
# y_test = test[target]

# model = LinearRegression()

# model.fit(x_train, y_train)
# y_pred = model.predict(x_train)
# train_mae = mean_absolute_error(y_train, y_pred)
# print('Training MAE: ${0:,.2f}'.format(train_mae))

# y_pred = model.predict(x_test)
# testma_error = mean_absolute_error(y_test, y_pred)
# print('Testing MAE: ${0:,.2f}'.format(testma_error))
# fitting_predicting(x_train, x_test, x_train_select_features, 'SALE_PRICE')


# for testing (this is prob it)

# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import numpy as np

# def getting_metrics_testing(testing_set, target):
  
#   # Mean absolute error for Testing
#   testing_target = testing_set[target]
#   y_testing_prediction = [baseline_initial] * len(train_target) # this step is necessary for function to work (same length) - this is our target repeated (the predicted value)
#   mae_test = mean_absolute_error(testing_target, y_testing_prediction) # testing target is the actual value
#   print(f"Our testing model's MA error is: ${mae_test:,.0f}")

#   # Mean squared error for Testing
#   squared_test_error = np.sqrt(mean_squared_error(testing_target, y_testing_prediction))
#   print(f"Our Testing model's squared error is: ${squared_test_error:,.0f}")

#   # r^2 score for Testing
#   r_testing_score = r2_score(testing_target, y_testing_prediction)
#   print('R-score Testing error is: {0:.4f}.'.format(r_testing_score))

# getting_metrics_testing(df, 'SALE_PRICE')

In [0]:
df_copy.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [0]:
df_copy.describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,23040.0,4.459116e+03,3.713270e+03,1.0,1341.0,3546.0,6673.75,16350.0
LOT,23040.0,3.539485e+02,6.280253e+02,1.0,22.0,49.0,375.00,9057.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,23039.0,1.079119e+04,1.103414e+03,0.0,10306.0,11211.0,11360.00,11697.0
RESIDENTIAL_UNITS,23039.0,1.827423e+00,1.009043e+01,0.0,0.0,1.0,2.00,750.0
COMMERCIAL_UNITS,23039.0,2.865142e-01,5.778224e+00,-148.0,0.0,0.0,0.00,570.0
TOTAL_UNITS,23039.0,2.320891e+00,1.207055e+01,0.0,1.0,1.0,2.00,755.0
GROSS_SQUARE_FEET,23039.0,3.508555e+03,2.218610e+04,0.0,494.0,1356.0,2280.00,1303935.0
YEAR_BUILT,23005.0,1.819672e+03,4.883766e+02,0.0,1920.0,1940.0,1965.00,2019.0
TAX_CLASS_AT_TIME_OF_SALE,23040.0,1.624479e+00,8.038978e-01,1.0,1.0,1.0,2.00,4.0


In [0]:
df_copy.describe(include=['O']).T

,count,unique,top,freq
BOROUGH,23040,5,4,7494
NEIGHBORHOOD,23040,252,FLUSHING-NORTH,685
BUILDING_CLASS_CATEGORY,23040,44,01 ONE FAMILY DWELLINGS,5061
TAX_CLASS_AT_PRESENT,23039,10,1,11071
BUILDING_CLASS_AT_PRESENT,23039,146,D4,3408
ADDRESS,23040,22691,100 JEROME STREET,6
APARTMENT_NUMBER,5201,1724,4,99
LAND_SQUARE_FEET,22987,3652,0,7500
BUILDING_CLASS_AT_TIME_OF_SALE,23040,147,D4,3408
SALE_DATE,23040,120,01/24/2019,480


In [0]:
# dropped some and reduced cardinality in others. Not doing that crashes the session.

top_land = df_copy['LAND_SQUARE_FEET'].value_counts()[:35].index
df_copy.loc[~df_copy['LAND_SQUARE_FEET'].isin(top_land), 'LAND_SQUARE_FEET'] = 'N/A'

building_class = df_copy['BUILDING_CLASS_CATEGORY'].value_counts()[:30].index
df_copy.loc[~df_copy['BUILDING_CLASS_CATEGORY'].isin(building_class), 'BUILDING_CLASS_CATEGORY'] = 'N/A'

bc_at_present = df_copy['BUILDING_CLASS_AT_PRESENT'].value_counts()[:30].index
df_copy.loc[~df_copy['BUILDING_CLASS_AT_PRESENT'].isin(bc_at_present), 'BUILDING_CLASS_AT_PRESENT'] = 'N/A'

In [0]:
features_nulls_high_cardinality = ['APARTMENT_NUMBER', 'EASE-MENT', 'ADDRESS']
df_copy = df_copy.drop(features_nulls_high_cardinality, axis=1)

In [0]:
df_copy.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,R4,10011.0,1.0,0.0,1.0,N/A,1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,N/A,10018.0,0.0,6.0,6.0,N/A,15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,N/A,10018.0,0.0,7.0,7.0,N/A,11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,R4,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,R1,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [0]:
matrix_features = df_copy.drop(columns="SALE_PRICE")
vector_target = df_copy['SALE_PRICE']

In [0]:
# cutoff = '2019-03-31'
# train_2 = df_copy[df_copy['SALE_DATE'] < cutoff]
# test_2 = df_copy[df_copy['SALE_DATE'] > cutoff]

In [0]:
# train/test split

from sklearn.model_selection import train_test_split

# X_train = train[features]
# y_train = train[target]
# X_test = test[features]
# y_test = test[target]

x_train, x_test, y_train, y_test = train_test_split(matrix_features, vector_target, random_state=20)

In [0]:
filling_nulls(x_test)
filling_nulls(x_train)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [0]:
# encoding once more

import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
x_train = encoder.fit_transform(x_train) # Fit to data, then transform it.
x_test = encoder.transform(x_test) # applying FE to both

### Ridge

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

model_2 = LinearRegression()

# Training error
model_2.fit(x_train, y_train)
y_pred = model_2.predict(x_train)
train_mae = mean_absolute_error(y_train, y_pred)
print('Training MAE: ${0:,.2f}'.format(train_mae))

# Testing error
y_pred = model_2.predict(x_test)
testma_error = mean_absolute_error(y_test, y_pred)
print('Testing MAE: ${0:,.2f}'.format(testma_error))

print(mean_squared_error(y_train, model_2.predict(x_train))) # original error
print(mean_squared_error(y_test, model_2.predict(x_test))) # testing error (overfitting)

Training MAE: $1,597,876.17
Testing MAE: $1,655,632.09
36254664371998.42
45908424189218.21


In [0]:
from sklearn.linear_model import Ridge 

model_3 = Ridge(alpha=20)

model_3.fit(x_train, y_train)
msquare_error = mean_squared_error(y_train, model_3.predict(x_test))
print(msquare_error)